In [ ]:
import logging
from binance.um_futures import UMFutures
from binance.lib.utils import config_logging

In [ ]:
import datetime
# datetime_str = '2022-01-01 00:00:00'

def get_unix_epoch_time_ms(datetime_str):
    datetime_obj = datetime.datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
    unix_epoch_time_ms = int(datetime_obj.timestamp() * 1000)
    return unix_epoch_time_ms

In [ ]:
now = datetime.datetime.now()
delta = datetime.timedelta(days=30)
stamps = []
for i in range(1, 85):
    dt = now - datetime.timedelta(minutes=i*500)
    if dt < now - delta:
        break
    stamps.append(dt.strftime('%Y-%m-%d %H:%M:%S'))


In [ ]:

stamps = stamps[::-1]
pairs = [[stamps[i], stamps[i+1]] for i in range(0, len(stamps), 2)]
pairs

https://binance-docs.github.io/apidocs/futures/en/#open-interest

In [ ]:
um_futures_client = UMFutures()
resp_data = []
for pair in pairs:
    try:
        s, e = pair
        print(f"s: {s}, e: {e}")
        start = get_unix_epoch_time_ms(s)
        end = get_unix_epoch_time_ms(e)
        print(f"start: {start}, end: {end}")
        resp = um_futures_client.open_interest_hist("BTCUSDT", "5m", startTime=start, endTime=end)
        resp_data.extend(resp)  
    except Exception as e:
        print(e)
        
resp_data    

In [ ]:
import pyperclip as clip

clip.copy(f"{resp_data[0]}")

In [ ]:
import pandas as pd

df = pd.DataFrame(resp_data)
df['sumOpenInterest'] = df['sumOpenInterest'].astype(float)
df

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)
df

In [ ]:
# Normalize sumOpenInterest by dividing it by its maximum value
df['normalized_sumOpenInterest'] = df['sumOpenInterest'] / df['sumOpenInterest'].max()
df

In [ ]:
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource



source = ColumnDataSource(df)
p = figure(x_axis_type='datetime', title='chart', width=1200, height=800)
p.vbar(x='timestamp', top='normalized_sumOpenInterest', source=source, width=0.5, color='blue')

# Rotate x-axis labels vertically
p.xaxis.major_label_orientation = "vertical"
# save the output to an HTML file
output_file('./charts/bar-open-interest-30-days-1m.html')
# save the plot to the HTML file
save(p)


In [ ]:
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource

source = ColumnDataSource(df)
p = figure(x_axis_type='datetime', title='chart', width=1200, height=800)
p.line(x='timestamp', y='sumOpenInterest', source=source, line_width=2, color='blue')

 # save the output to an HTML file
output_file(f'./charts/open-interest-30-days-1m.html')
# save the plot to the HTML file
save(p)